In [36]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

Merge CDE with ACS data

In [37]:
CDS_data = pd.read_pickle("Data/SchoolDirectory/CDE_data.pkl")

In [38]:
zip_data = pd.read_pickle("Data/ACS/ACS_data.pkl")

In [39]:
CDS_data['zipcode1'] = CDS_data['Zip'].str[:5]
CDS_data['zipcode2'] = np.where(CDS_data['zipcode1'] == "No Da", "", CDS_data['zipcode1'])
CDS_data['zipcode'] = CDS_data['zipcode2'].astype(float)
zip_data['zipcode'] = zip_data['zipcode'].astype(float) 
Charter_Schools_Analysis = CDS_data.merge(zip_data, left_on='zipcode', right_on='zipcode')

Charter_Schools_Analysis = Charter_Schools_Analysis.loc[Charter_Schools_Analysis['ANN_TOTAL_ENROLL'] > 0]

Charter_Schools_Analysis['openyear'] = Charter_Schools_Analysis['OpenDate'].astype(str).str[:4]
Charter_Schools_Analysis['openyear'] = Charter_Schools_Analysis['openyear'].astype(int)

Merge with GreatSchools Data

In [40]:
School_Neighbors = pd.read_csv("Data/GreatSchools/School_Neighbors.csv")
School_Neighbors['CDS_CODE'] = School_Neighbors['CDS_CODE'].astype(str)
School_Neighbors['CDS_CODE'] = School_Neighbors['CDS_CODE'].str.zfill(14)

In [41]:
Charter_Schools_Analysis = pd.merge(Charter_Schools_Analysis, School_Neighbors, on=['CDS_CODE'], how = 'outer')
Charter_Schools_Analysis = Charter_Schools_Analysis.loc[Charter_Schools_Analysis['ANN_TOTAL_ENROLL'] > 0]

Impute missing ELL scores and Great Schools using overall average of each individual school

In [42]:
Charter_Schools_One_School = Charter_Schools_Analysis.copy()
Charter_Schools_One_School['First_School'] = (Charter_Schools_One_School.groupby('School_x').cumcount() == 0).astype(int)
Charter_Schools_One_School = Charter_Schools_One_School.loc[Charter_Schools_One_School['First_School'] > 0]

In [43]:
average_ELL = Charter_Schools_One_School['Pct_ELL'].mean()
average_FRPM = Charter_Schools_One_School['Pct_FRPM'].mean()
average_PS_score = Charter_Schools_Analysis['neighborhood_PS_score'].mean()

In [44]:
Charter_Schools_Analysis['Pct_ELL2'] = np.where(Charter_Schools_Analysis['Pct_ELL'].isnull(), average_ELL, Charter_Schools_Analysis.Pct_ELL)
Charter_Schools_Analysis['Pct_FRPM2'] = np.where(Charter_Schools_Analysis['Pct_FRPM'].isnull(), average_FRPM, Charter_Schools_Analysis.Pct_FRPM)
Charter_Schools_Analysis['neighborhood_PS_score2'] = np.where(Charter_Schools_Analysis['neighborhood_PS_score'].isnull(), average_PS_score, Charter_Schools_Analysis.neighborhood_PS_score)

In [45]:
missing_ell = Charter_Schools_Analysis[Charter_Schools_Analysis['Pct_ELL2'].isnull()]
missing_FRPM = Charter_Schools_Analysis[Charter_Schools_Analysis['Pct_FRPM2'].isnull()]
missing_PS_score = Charter_Schools_Analysis[Charter_Schools_Analysis['neighborhood_PS_score2'].isnull()]

In [46]:
Charter_Schools_Analysis = Charter_Schools_Analysis.drop(['Pct_ELL', 'Pct_FRPM', 'neighborhood_PS_score'], axis=1)
Charter_Schools_Analysis = Charter_Schools_Analysis.rename(columns={'Pct_ELL2': 'Pct_ELL', 'Pct_FRPM2': 'Pct_FRPM', 'neighborhood_PS_score2': 'PS_score'})

In [47]:
Charter_Schools_Analysis['Pct_FRPM']  = np.where(Charter_Schools_Analysis['Pct_FRPM'] > 1, average_FRPM, Charter_Schools_Analysis['Pct_FRPM'])
Charter_Schools_Analysis['Pct_ELL']  = np.where(Charter_Schools_Analysis['Pct_ELL'] > 1, average_FRPM, Charter_Schools_Analysis['Pct_ELL'])


In [48]:
#missing_ell1 = Charter_Schools_Analysis[Charter_Schools_Analysis['Pct_ELL'].isnull()]
#missing_FRPM1 = Charter_Schools_Analysis[Charter_Schools_Analysis['Pct_FRPM'].isnull()]
#missing_PS = Charter_Schools_Analysis[Charter_Schools_Analysis['PS_score'].isnull()]

Prepare for analysis

In [49]:
Charter_Schools_Analysis = Charter_Schools_Analysis.drop(columns=['Unnamed: 0', 'School_y',  'Latitude_y', 'Longitude_y', 'public_schools_in_range_gs_weight',
                                                                    'public_schools_in_range_ps_weight', 'public_schools_in_range_total', 'neighborhood_GS_score',   
                                                                     'public_schools_in_range_gs_weight', 'public_schools_in_range_ps_weight', 'public_schools_in_range_total',
                                                                     'neighborhood_GS_score'
                                                                   ])
                                                                    

Charter_Schools_Analysis.to_pickle("Data/AnalysisSet/Charter_Schools_Analysis.pkl")